In [ ]:
! pip install pyspark==3.3.1

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

In [ ]:
df = spark.read.json("/home/airflow/workspaces/airflow-spark/data/bronze/twitter_datascience")

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql import functions as f

In [ ]:
df.select(f.explode("data")).printSchema()

In [ ]:
df.select(f.explode("data")).show()

In [ ]:
df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

In [ ]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [ ]:
tweet_df.show(5)

In [ ]:
df.printSchema()

In [ ]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

In [ ]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [ ]:
user_df.show(5)

In [ ]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')